In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dolfin as df

In [3]:
def delete_dataset(store, dataset_name):
    """ Should be used with care, especially when considering MPI."""
    import h5py
    if store.hdf.has_dataset(dataset_name):
        with h5py.File(store.filepath, 'a') as f:
            print("Deleting dataset", dataset_name, "from file", store.filepath)
            del f[dataset_name]

In [10]:
from pantarei.fenicsstorage import FenicsStorage
from ..multidiffusion_model import get_default_coefficients


phi = [0.14, 0.05]
phi_tot = sum(phi)
if __name__ == "__main__":
    compartments = ["ecs", "pvs"]
    data_file = "../DATA/PAT_002/FENICS/cdata_32.hdf"
    storage = FenicsStorage(data_file, "a")
    delete_dataset(storage, "/test")
    timevec = storage.read_timevector("multidiffusion")
    u = storage.read_function("multidiffusion", idx=0)
#     el = storage.read_element("cdata")
#     domain = storage.read_domain()

#     W = df.FunctionSpace(domain, el)
    c = df.Function(u.function_space().sub(0).collapse())
    for idx, ti in enumerate(timevec):
        u = storage.read_checkpoint(u, "multidiffusion", idx=idx)
        c.vector()[:] = sum([phi[j] * uj.vector() for j, uj in enumerate(u.split(deepcopy=True))])

        storage.write_checkpoint(c, "/test", ti)

storage.close()

In [25]:
storage = FenicsStorage(data_file, "a")
delete_dataset(storage, "/multidiffusion_total")
storage.close()

In [11]:
file = FenicsStorage(storage.filepath, "r")
file.to_xdmf("test", "test")
file.close()

In [15]:
c.vector().size()

183136

In [18]:
vec = c.vector()

In [ ]:
st

In [22]:
vec.get_local()

array([0.        , 0.        , 0.05712067, ..., 0.        , 0.06327271,
       0.        ])

In [21]:
all(vec.get_local() == vec)

True